In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import json

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    roc_auc_score
)

import warnings
warnings.filterwarnings("ignore")


# Target

In [2]:
def get_target(input_df, ticker):
    df = input_df.copy()
    df["Target"] = (df[f"Close_{ticker}"].shift(-1) > df[f"Close_{ticker}"]).astype(int)
    df.dropna(inplace=True)
    return df

# Model: Ensemble (Voting Soft)

In [3]:
def build_ensemble_model():
    lr_pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("lr", LogisticRegression(max_iter=2000, random_state=42))
    ])

    rf_pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("rf", RandomForestClassifier(
            n_estimators=100,
            max_depth=4,
            min_samples_leaf=10,
            random_state=42,
            n_jobs=-1
        ))
    ])

    svm_pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("svc", SVC(
            kernel="rbf",
            C=1.0,
            gamma="scale",
            probability=True,
            random_state=42
        ))
    ])

    xgb_pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("xgb", XGBClassifier(
            n_estimators=50,
            max_depth=3,
            learning_rate=0.1,
            random_state=42,
            n_jobs=-1,
            eval_metric="logloss"
        ))
    ])

    model = VotingClassifier(
        estimators=[
            ("lr", lr_pipeline),
            ("rf", rf_pipeline),
            ("svm", svm_pipeline),
            ("xgb", xgb_pipeline),
        ],
        voting="soft"
    )
    return model

# Walk Forward validation

In [4]:
def walk_forward_validation(
    df,
    features,
    target_col="Target",
    date_col="DATE",
    start_year=2010,
    first_train_end_year=2015,
    last_test_year=2023
):
    """
    Train: start_year -> train_end_year
    Test : train_end_year+1
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col).reset_index(drop=True)

    all_y_true = []
    all_y_pred = []
    all_y_proba = []

    fold_rows = []

    for train_end_year in range(first_train_end_year, last_test_year):
        test_year = train_end_year + 1

        train_mask = (df[date_col].dt.year >= start_year) & (df[date_col].dt.year <= train_end_year)
        test_mask = (df[date_col].dt.year == test_year)

        train_df = df[train_mask]
        test_df = df[test_mask]

        # jeżeli jakiś rok nie ma danych to skip
        if len(train_df) < 200 or len(test_df) < 50:
            continue

        X_train = train_df[features]
        y_train = train_df[target_col]

        X_test = test_df[features]
        y_test = test_df[target_col]

        model = build_ensemble_model()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]  # ważne dla ROC-AUC

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        auc = roc_auc_score(y_test, y_proba)

        fold_rows.append({
            "train_end_year": train_end_year,
            "test_year": test_year,
            "n_train": len(train_df),
            "n_test": len(test_df),
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "roc_auc": auc
        })

        all_y_true.extend(y_test.tolist())
        all_y_pred.extend(y_pred.tolist())
        all_y_proba.extend(y_proba.tolist())

    folds_df = pd.DataFrame(fold_rows)

    return folds_df, np.array(all_y_true), np.array(all_y_pred), np.array(all_y_proba)




# Block bootstrap

In [5]:
def block_bootstrap_accuracy(y_true, y_pred, block_size=20, n_bootstrap=1000, random_state=42):
    """
    Bootstrap na wynikach testowych (y_true/y_pred), losowanie blokami.
    """
    rng = np.random.default_rng(random_state)
    n = len(y_true)

    if n < block_size:
        raise ValueError("Za mało danych do bootstrapa w tej konfiguracji.")

    acc_samples = []

    for _ in range(n_bootstrap):
        sampled_idx = []

        while len(sampled_idx) < n:
            start = rng.integers(0, n - block_size + 1)
            block = list(range(start, start + block_size))
            sampled_idx.extend(block)

        sampled_idx = sampled_idx[:n]
        y_true_bs = y_true[sampled_idx]
        y_pred_bs = y_pred[sampled_idx]

        acc = accuracy_score(y_true_bs, y_pred_bs)
        acc_samples.append(acc)

    acc_samples = np.array(acc_samples)
    ci_low = np.percentile(acc_samples, 2.5)
    ci_high = np.percentile(acc_samples, 97.5)

    return acc_samples, ci_low, ci_high



# Walk_forward + Bootstrap

In [6]:
def run_stage4_for_ticker(df_raw, ticker, selected_features):
    df = get_target(df_raw, ticker)

    # upewnij się że feature istnieją
    selected_features = [f for f in selected_features if f in df.columns]

    folds_df, y_true_all, y_pred_all, y_proba_all = walk_forward_validation(
        df=df,
        features=selected_features,
        target_col="Target",
        date_col="DATE",
        start_year=2010,
        first_train_end_year=2015,
        last_test_year=2023  # ustaw pod swój dataset
    )

    print("=" * 60)
    print(f" WALK-FORWARD RESULTS for {ticker}")
    print(folds_df)

    print("\n--- Summary ---")
    print("Mean accuracy:", folds_df["accuracy"].mean())
    print("Std  accuracy:", folds_df["accuracy"].std())
    print("Mean roc_auc :", folds_df["roc_auc"].mean())

    # Block Bootstrap
    acc_samples, ci_low, ci_high = block_bootstrap_accuracy(
        y_true=y_true_all,
        y_pred=y_pred_all,
        block_size=20,
        n_bootstrap=1000
    )

    print("\n" + "=" * 60)
    print(f" BLOCK BOOTSTRAP for {ticker}")
    print(f"95% CI accuracy: [{ci_low:.4f}, {ci_high:.4f}]")
    print(f"Bootstrap mean accuracy: {acc_samples.mean():.4f}")

    return folds_df, acc_samples


# Example use

In [12]:

tickers = ["AAPL", "GOOGL", "MSFT"]
measurments = ["accuracy", "precision", "recall", "roc_auc"]

with open("../../models_results/feature_dict.json", "r") as f:
    feature_dict = json.load(f)


def get_all_features(df):
    return [c for c in df.columns if c not in ["DATE", "index", "Target"]]

for share in tickers:
    data = pd.read_csv(f"../../data/all_data/all_{share}_data.csv")
    df_tmp = get_target(data, share)
    features_aapl = get_all_features(df_tmp)
    print(f"------{share}-----")
    for stat in measurments:
        selected_features = feature_dict[share][stat]
        print(f"Selected features for {share} based on {stat}:")
        # print(f"Running for {share} with all features ({len(features_aapl)})")
        # run_stage4_for_ticker(data, share, features_aapl)
        print(f"Running for {share} with selected features ({len(selected_features)})")
        run_stage4_for_ticker(data, share, selected_features)

------AAPL-----
Selected features for AAPL based on accuracy:
Running for AAPL with selected features (11)
 WALK-FORWARD RESULTS for AAPL
   train_end_year  test_year  n_train  n_test  accuracy  precision    recall  \
0            2015       2016     1510     252  0.515873   0.522822  0.947368   
1            2016       2017     1762     251  0.482072   0.571429  0.231884   
2            2017       2018     2013     251  0.482072   0.495798  0.457364   
3            2018       2019     2264     252  0.559524   0.575000  0.938776   
4            2019       2020     2516     253  0.553360   0.566667  0.744526   
5            2020       2021     2769     252  0.563492   0.552174  0.947761   
6            2021       2022     3021     251  0.474104   0.468468  0.881356   
7            2022       2023     3272     250  0.548000   0.595745  0.600000   

    roc_auc  
0  0.493334  
1  0.515775  
2  0.499238  
3  0.502883  
4  0.572238  
5  0.541740  
6  0.504715  
7  0.556558  

--- Summary --